<a href="https://colab.research.google.com/github/koroltony/pyrtltest/blob/main/PyRTLAdder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Pyrtl in Colab

In [1]:
!pip install pyrtl


## Build Circuit in Pyrtl

In [2]:
import pyrtl

# Define your PyRTL circuit
def one_bit_add(a, b, carry_in):
    assert len(a) == len(b) == 1
    sum_ = a ^ b ^ carry_in
    carry_out = a & b | a & carry_in | b & carry_in
    return sum_, carry_out

def ripple_add(a, b, carry_in=0):
    a, b = pyrtl.match_bitwidth(a, b)
    if len(a) == 1:
        sumbits, carry_out = one_bit_add(a, b, carry_in)
    else:
        lsbit, ripplecarry = one_bit_add(a[0], b[0], carry_in)
        msbits, carry_out = ripple_add(a[1:], b[1:], ripplecarry)
        sumbits = pyrtl.concat(msbits, lsbit)
    return sumbits, carry_out

# Instantiate an adder into a 3-bit counter
counter = pyrtl.Register(bitwidth=3, name='counter')
sum_, carry_out = ripple_add(counter, pyrtl.Const("1'b1"))
counter.next <<= sum_

# Simulate the instantiated design for 15 cycles
sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace)
for cycle in range(15):
    sim.step({})
sim_trace.render_trace()
print("not the cleanest output, so we need to run the waveform simulation in the next cell")

#create verilog file
with open('hardware.v', 'w') as file:
    pyrtl.output_to_verilog(file, add_reset=True, block=None)

        ▏0                   ▏5                   ▏10                 

counter 0x0 ╳0x1╳0x2╳0x3╳0x4 ╳0x5╳0x6╳0x7╳0x0╳0x1 ╳0x2╳0x3╳0x4╳0x5╳0x6

not the cleanest output, so we need to run the waveform simulation in the next cell


## Waveform simulation

In [3]:
#@title Waveform Viewer
import json
from IPython.display import IFrame

def generate_waveform_data(sim_trace, hex_output=False):
    waveform_signals = []

    for signal_name, signal_values in sim_trace.trace.items():
        waveform_signal = {
            "name": signal_name,
            "wave": "2" * len(signal_values),
            "data": [hex(value) if hex_output else str(value) for value in signal_values]
        }
        waveform_signals.append(waveform_signal)
    #visual clock signal on WaveDrom
    waveform_clock = {
      'name': 'clock',
      'wave': 'p' * len(signal_values)
    }
    waveform_signals.append(waveform_clock)

    waveform_data = {
        "signal": waveform_signals,
        "foot": {
            "tick": 0
        }
    }
    return waveform_data

# Generate waveform data (Choose hex true or false)
waveform_data_hex = generate_waveform_data(sim.tracer, hex_output=True)
waveform_data_decimal = generate_waveform_data(sim.tracer, hex_output=False)

# Save the waveform data to a JSON file (to open in WaveDrom website)
with open('waveform_hex.json', 'w') as file:
    json.dump(waveform_data_hex, file, indent=2)

with open('waveform_dec.json', 'w') as file:
    json.dump(waveform_data_decimal, file, indent=2)

print("Waveforms saved to 'waveform_---.json'")
print("-> Copy paste the contents from either of the JSON files into the text-box below")
print("   (click on the folder icon on the left of the screen to see the files)")
print("-> Make sure to clear the original text in the text-box below, as it may be from previous circuits")
print("-> Note: the waveform is offset due to real-life delay in hardware")

# Display the WaveDrom website in Colab
IFrame(src='https://wavedrom.com/editor.html', width='100%', height='500px')


Waveforms saved to 'waveform_---.json'
-> Copy paste the contents from either of the JSON files into the text-box below
   (click on the folder icon on the left of the screen to see the files)
-> Make sure to clear the original text in the text-box below, as it may be from previous circuits
-> Note: the waveform is offset due to real-life delay in hardware
